# vcu 

> vcu torque table generation 

In [ ]:
#| default_exp config.vcu

    
# introduction

For real vcu, values in the table will be the requrested torque
Current throttlel (0,1) should be a coefficient of multplicative factor
like between +/- 20% or empirically give safety bounds.
action space will be then within the bounds
TODO ask for safety bounds and real vcu to be integrated.
TODO generate a mask according to WLTC to reduce parameter optimization space.


In [ ]:
#| export

import glob
import os
import os.path
from pathlib import Path
import numpy as np
import pandas as pd  # type: ignore
from scipy import interpolate  # type: ignore

In [ ]:
#| export
def generate_torque_table(
    pedal_scale: tuple, speed_scale: tuple  # pedal range (0,1)  # speed range (0,120)
):
    """
    Generate VCU calibration parameters for a given truck.
    simple piecewise linear surface, close to default calibration table
    Input is npd 17, nvl 21; output vcu_param_list as float32
    pedal is x(column), velocity is y(row) )

    Parameters:

        pedal_scale: tuple
            pedal range (0,1)
        speed_scale: tuple
            speed range (0,120)

    Return:
        pandas dataframe
    """

    pdv, vlv = np.meshgrid(list(pedal_scale), list(speed_scale), sparse=True)
    v = pdv / (1 + np.sqrt(np.abs(vlv)))
    pd_v = pd.DataFrame(v, columns=list(pedal_scale), index=list(speed_scale))

    return pd_v

In [ ]:
#| export
def generate_vcu_calibration(
    npd: int,  # number of pedal steps
    pedal_range: tuple,  # pedal range (0,1)
    nvl: int,  # number of velocity steps
    velocity_range: tuple,  # speed range (0,120)
    shortcut: int,  # 1: use default eco calibration table
    data_root: Path,  # path to data folder
):
    """
    Generate VCU calibration parameters for a given truck.

    pedal is x(column), velocity is y(row)
    input : npd 17, nvl 21; output vcu_param_list as float32

    Parameters:

        npd: int
            number of pedal steps
        pedal_range: tuple
            pedal range (0,1)
        nvl: int
            number of velocity steps
        velocity_range: tuple
            speed range (0,120)
        shortcut: int
            1: use segment-wise linear eco calibration table
            2: use init table
            3: use latest pedal map that was used
        data_root: str
            path to data folder

    Return:
        pandas dataframe
    """
    ped = np.linspace(pedal_range[0], pedal_range[1], num=npd)  # 0 - 100% pedal

    if shortcut == 1:
        vel_ = np.linspace(
            velocity_range[0], velocity_range[1], num=nvl - 1
        )  # 0 - 120 kmph velocity
        vel = np.insert(vel_, 1, 7)  # insert 7 kmph, and convert to m/s
        pdv, vlv = np.meshgrid(ped, vel / 3.6, sparse=True)
        v = pdv / (1 + np.sqrt(np.abs(vlv)))
        pd_v = pd.DataFrame(v, columns=ped, index=vel)

    elif shortcut == 2:  # import default eco calibration table
        table_path = data_root.joinpath(
            "vb7_init_table.csv"
        )  # init table is driver independent in the pardir.
        pd_v = pd.read_csv(table_path, header=0, index_col=0)
        # table_path = datafolder + "/54_vertices_approx-regen3.csv"  # init table is driver independent in the pardir.
        # pd_data = pd.read_csv(table_path, header=0, index_col=0)
    elif shortcut == 3:  # import latest pedal map that was used
        files = glob.glob(str(data_root) + "/last_table*.csv")
        if not files:  # files is empty list []
            print("no last table is available. Get init table instead.")
            latest_table = Path(__file__).parent.joinpath(
                "vb7_init_table.csv"
            )  # init table is driver independent in the pardir.
        else:
            latest_table = max(files, key=os.path.getmtime)
        # latest = datafolder + "last_table.csv"  # init table is driver relevant.
        pd_v = pd.read_csv(latest_table, header=0, index_col=0)

    else:
        vel = np.ones(nvl)
        pdv, vlv = np.meshgrid(ped, vel, sparse=False)
        pd_v = pd.DataFrame(pdv, columns=ped, index=vel)
    return pd_v

In [ ]:
# fig, ax = plt.subplots(subplot_kw={"projection": "3d"})
# # Plot the surface.
# surf = ax.plot_surface(pdv, vlv, v, cmap=cm.coolwarm,
#                        linewidth=0, antialiased=False)
# # Customize the z axis.
# ax.set_zlim(-0.01, 1.01)
# ax.zaxis.set_major_locator(LinearLocator(10))
# # A StrMethodFormatter is used automatically
# ax.zaxis.set_major_formatter('{x:.02f}')
# 
# # Add a color bar which maps values to colors.
# fig.colorbar(surf, shrink=0.5, aspect=5)
# plt.show()
# return np.float32(v)

In [ ]:
#| export
def generate_lookup_table(
    pedal_range: tuple, velocity_range: tuple, calib_table: pd.DataFrame
):
    """
    Generate VCU calibration parameters for a given truck.

    pedal in x(col), velocity in y(row)
    input : npd 17, nvl 21; output vcu_param_list as float32

    Parameters:

        pedal_range: tuple
            pedal range (0,1)
        velocity_range: tuple
            speed range (0,120)
        calib_table: pandas dataframe
            calibration table

    Return:

        numpy array
    """
    nvl, npd = calib_table.shape
    p_step = complex(0, npd)
    v_step = complex(0, nvl)
    grid_v, grid_p = np.mgrid[
        velocity_range[0] : velocity_range[1] : v_step,
        pedal_range[0] : pedal_range[1] : p_step,
    ]
    calib_lookup = interpolate.RegularGridInterpolator(
        (grid_p, grid_v), calib_table.to_numpy()
    )
    return calib_lookup

## testing

In [ ]:
#| hide
from fastcore.test import *

In [ ]:
#| export
def test_generate_lookup_table():
    vcu_calib_table_row = 17  # number of pedal steps
    vcu_calib_table_col = 21  # numnber of velocity steps
    pedal_range = [0, 1.0]
    velocity_range = [0, 120.0]
    vcu_calib_table = generate_vcu_calibration(
        npd=vcu_calib_table_row,
        pedal_range=pedal_range,
        nvl=vcu_calib_table_col,
        velocity_range=velocity_range,
        shortcut=1,
        data_root=Path("."),
    )
    vcu_lookup_table = generate_lookup_table(
        pedal_range, velocity_range, vcu_calib_table
    )
    return vcu_lookup_table


def test_generate_vcu_calibration():
    vcu_calib_table_row = 17  # number of pedal steps
    vcu_calib_table_col = 21  # numnber of velocity steps
    pedal_range = [0, 1.0]
    velocity_range = [0, 20.0]
    vcu_calib_table = generate_vcu_calibration(
        npd=vcu_calib_table_row,
        pedal_range=pedal_range,
        nvl=vcu_calib_table_col,
        velocity_range=velocity_range,
        shortcut=1,
        data_root=Path("."),
    )
    return vcu_calib_table

In [ ]:
#| output: true
vcu_calib_table = test_generate_vcu_calibration()
vcu_calib_table

,0.0000,0.0625,0.1250,0.1875,0.2500,0.3125,0.3750,0.4375,0.5000,0.5625,0.6250,0.6875,0.7500,0.8125,0.8750,0.9375,1.0000
0.000000,0.0,0.062500,0.125000,0.187500,0.250000,0.312500,0.375000,0.437500,0.500000,0.562500,0.625000,0.687500,0.750000,0.812500,0.875000,0.937500,1.000000
7.000000,0.0,0.026102,0.052204,0.078307,0.104409,0.130511,0.156613,0.182715,0.208818,0.234920,0.261022,0.287124,0.313227,0.339329,0.365431,0.391533,0.417635
1.052632,0.0,0.040565,0.081130,0.121695,0.162260,0.202825,0.243390,0.283955,0.324520,0.365085,0.405650,0.446215,0.486780,0.527345,0.567910,0.608475,0.649040
2.105263,0.0,0.035416,0.070833,0.106249,0.141666,0.177082,0.212498,0.247915,0.283331,0.318748,0.354164,0.389580,0.424997,0.460413,0.495830,0.531246,0.566662
3.157895,0.0,0.032273,0.064547,0.096820,0.129093,0.161366,0.193640,0.225913,0.258186,0.290460,0.322733,0.355006,0.387280,0.419553,0.451826,0.484099,0.516373
4.210526,0.0,0.030027,0.060054,0.090080,0.120107,0.150134,0.180161,0.210187,0.240214,0.270241,0.300268,0.330294,0.360321,0.390348,0.420375,0.450402,0.480428
5.263158,0.0,0.028292,0.056583,0.084875,0.113167,0.141459,0.169750,0.198042,0.226334,0.254625,0.282917,0.311209,0.339501,0.367792,0.396084,0.424376,0.452667
6.315789,0.0,0.026887,0.053774,0.080661,0.107549,0.134436,0.161323,0.188210,0.215097,0.241984,0.268871,0.295758,0.322646,0.349533,0.376420,0.403307,0.430194
7.368421,0.0,0.025713,0.051426,0.077140,0.102853,0.128566,0.154279,0.179992,0.205706,0.231419,0.257132,0.282845,0.308558,0.334272,0.359985,0.385698,0.411411
8.421053,0.0,0.024709,0.049418,0.074127,0.098836,0.123545,0.148254,0.172963,0.197672,0.222381,0.247090,0.271799,0.296509,0.321218,0.345927,0.370636,0.395345


In [ ]:
# #| output: true
# vcu_lookup_table = test_generate_lookup_table()
# vcu_calib_table

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()